In [8]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

In [9]:
llm = Ollama(model="llama3.2", base_url="http://127.0.0.1:11434")

In [10]:
embed_model = OllamaEmbeddings(
    model="llama3.2",
    base_url='http://127.0.0.1:11434'
)

In [11]:
text = """
    In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
    """

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_text(text)

In [13]:
vector_store = Chroma.from_texts(chunks, embed_model)

In [14]:
retriever = vector_store.as_retriever()

In [15]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

In [16]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/media/santapong/DATA/Project/CapStone-Project/.venv/lib/python3.10/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

In [18]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [19]:
response = retrieval_chain.invoke({"input": "Tell me name of monkeys and where do they live"})
print(response['answer'])

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Based on the context, the names of the monkeys are:

1. Coco
2. Mango

They live in a tropical rainforest.
